### Imports & constants

In [ ]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt

import nibabel as nib

import utils

In [ ]:
VERSE2019_PATH = os.path.join(os.curdir, 'datasets', 'VerSe2019')
OUTPUT_PATH = os.path.join(os.curdir, 'output', 'VerSe2019.npz')

### Data processing

In [ ]:
imgs_s = [] # np.array with sagittal MIPs
imgs_f = [] # np.array with frontal MIPs
v_levels = [] # np.array with labels in mm
ids = [] # np.array with CTs identifiers
thicks = [] # np.array with CTs slices' thicknesses

In [ ]:
ct_paths = glob.glob(os.path.join(VERSE2019_PATH, '*', 'rawdata', '*', '*.nii.gz'))
json_paths = glob.glob(os.path.join(VERSE2019_PATH, '*', 'derivatives', '*', '*.json'))

In [ ]:
for i, ct_path in enumerate(ct_paths):
    print(i)

    # --- CT id naming convention ---
    name = ct_path.split('/')[-1].split('.')[0][:-3]
    dataset_type = ct_path.split('/')[3][15:] 
        
    if dataset_type == 'training':
        dataset_type = 'train'
    elif dataset_type == 'validation':
        dataset_type = 'val'
    
    ct_id = f'{name}_{dataset_type}'
    
    if ct_id in ids:
        print(f'{ct_id} already processed...')
        continue
    
    print(ct_id)
    
    # --- finding & loading corresponding .json ---
    json_path = [path for path in json_paths if name in path][0]
    # ctds_list = utils.load_centroids(json_path)
    ctds = utils.load_centroids(json_path)
    
    # --- preprocessing CT ---
    img = nib.load(ct_path)

    img_unsampled = utils.reorient_to(img, axcodes_to=('I', 'P', 'L')) # for calculating spacings

    # 1mm normalization + axes order normalization
    img = utils.reorient_to(img, axcodes_to=('I', 'P', 'L')) # CT in height x depth x width order
    ctds = utils.reorient_centroids_to(ctds, img)

    img = utils.resample_nib(img, voxel_spacing=(1, 1, 1), order=3)
    ctds = utils.rescale_centroids(ctds, img, (1, 1, 1))
     
    ctds_dict = {seq[0]: seq[1] for seq in ctds if seq[0] in (19, 22)}
    if not ctds_dict:
        print(f'No T12 & L3 for {name}...')
        continue
    
    slice_thickness = img.shape[0] / img_unsampled.shape[0]
    width_spacing = img.shape[2] / img_unsampled.shape[2]

    # getting np.array & transposing axes to assumed convention (depth x width x height)
    img = img.get_fdata()
    img = np.transpose(img, axes=[1, 2, 0])

    # converting to HU scale
    img += 24 # to be consistent with -1000 for air (-1024 in VerSe)
    
    # mass center calculation & 0ing everything outside body
    img, center_h, center_w = utils.get_mass_center(img)
        
    # frontal MIP
    img_cropped = utils.crop_ct(img, center_h, center_w, for_frontal=True)
    img_mip_f = np.amax(img_cropped, axis=0)
    img_mip_f = np.swapaxes(img_mip_f, 0, 1)

    # sagittal MIP
    try:
        img_cropped = utils.crop_ct(img, center_h, center_w, for_frontal=False, 
                                    sides_cut=40) # 40 instead of default 30
        img_mip_s = np.amax(img_cropped, axis=1) 
    except ValueError as e:
        print(e)
        print(f'Sagittal crop impossible for {ct_id} of shape {img.shape}...')
        continue
        
    img_mip_s = np.swapaxes(img_mip_s, 0, 1)

    # calculating labels in mm
    t12_lvl = -1 if not 19 in ctds_dict else int(ctds_dict[19])
    l3_lvl = -1 if not 22 in ctds_dict else int(ctds_dict[22])
    
    # saving
    imgs_s.append(img_mip_s)
    imgs_f.append(img_mip_f)
    v_levels.append([t12_lvl, l3_lvl])
    ids.append(ct_id)
    thicks.append(slice_thickness)

In [ ]:
# exporting to .npz
imgs_s = np.asarray(imgs_s)
imgs_f = np.asarray(imgs_f)
v_levels = np.asarray(v_levels)
ids = np.asarray(ids)
thicks = np.asarray(thicks)
n_mips = imgs_s.shape[0]

np.savez_compressed(OUTPUT_PATH, imgs_s=imgs_s, imgs_f=imgs_f,
                    v_levels=v_levels, ids=ids, thicks=thicks, n_mips=n_mips)